In [7]:
import cPickle
import itertools

with open("/Users/pushpendrerastogi/export/kbmvlsa/dbpedia.trecweb.make/field_tokens.pkl") as f:
    data = cPickle.load(f)

fields = ["DOCNO", "DOCHDR", "names", "category", "attributes", "SimEn", "RelEn"]
for idx, (vocab, field) in enumerate(itertools.izip(data, fields)):
    data[idx] = sorted(data[idx].iteritems(), key=lambda x: (x[1], x[0]), reverse=True)

In [1]:
%load_ext Cython

In [8]:
%%cython --cplus
# distutils: language=c++
from libcpp.vector cimport vector
from libcpp.pair cimport pair
from libcpp.string cimport string
from cython.operator cimport dereference as deref, postincrement
import numpy as np
cimport numpy as np
import zipfile
import os, tempfile
cdef class FieldedHitList:
    "A Cython class to store a HitList"
    def __cinit__(self, int n_fields, list field_to_vocabsize):
        self.n_fields = n_fields
        self.field_to_vocabsize = field_to_vocabsize
        for f, vocabsize in zip(range(n_fields), field_to_vocabsize):
            vec_ptr = new vector[vector[pair[int,int]]]()
            for v in range(vocabsize):
                vec2_ptr = new vector[pair[int,int]]()
                deref(vec_ptr).push_back(deref(vec2_ptr))
            self.field_token_doc_count.push_back(deref(vec_ptr))
        return

    cdef int update(self, int field_idx, int token_idx, int doc_idx):
        cdef pair[int,int]* topelem_ptr
        cdef vector[pair[int,int]]* ft_ptr = &(
            self.field_token_doc_count[field_idx][token_idx])
        cdef int retval
        if deref(ft_ptr).size():
            topelem_ptr = &(deref(ft_ptr).back())
            if deref(topelem_ptr).first == doc_idx:
                postincrement(deref(topelem_ptr).second)
                retval = 0
            else:
                deref(ft_ptr).push_back((doc_idx, 1))
                retval = 1
        else:
            deref(ft_ptr).push_back((doc_idx, 1))
            retval = 1
        return retval


    cdef int _serialize(self,
                        np.ndarray[np.uint32_t, mode="c", ndim=1]& arr,
                        string arcname,
                        zipf,
                        tmpfile_fn):
        ''' Save `array` to `zipf` with `arcname` in two steps.
        1st write `arr` to `tmpfile_fn`.
        2nd add the `tmpfile_fn` to `zipf` under the name `arcname`
        '''
        with open(tmpfile_fn, 'wb') as tmpfile:
            format.write_array(tmpfile, arr, allow_pickle=False)
        zipf.write(tmpfile_fn, arcname=arcname)
        return 1

    cdef int serialize(self, string filename):
        cdef int field_idx
        cdef np.ndarray indptr
        cdef np.ndarray indices
        cdef np.ndarray data
        cdef int data_size
        cdef int vocab_size
        cdef int data_idx
        cdef int offset_idx
        cdef vector[pair[int,int]]* column_ptr
        with zipfile.ZipFile(filename, mode="w", compression=zipfile.ZIP_STORED,
                             allowZip64=True) as zipf:
            for field_idx in range(self.n_fields):
                vocab_size = self.field_token_doc_count[field_idx].size()
                indptr = np.empty([vocab_siz,], dtype=np.uint32_t, mode="c")
                data_size = 0
                for token_idx in range(vocab_size):
                    indptr[token_idx] = self.field_token_doc_count[field_idx][token_idx].size()
                    data_size += indptr[token_idx]

                print field_idx, data_size
                indices = np.empty([data_size,], dtype=np.uint32_t)
                data = np.empty([data_size,], dtype=np.uint32_t)
                data_idx = 0
                for token_idx in range(vocab_size):
                    column_ptr = &self.field_token_doc_count[field_idx][token_idx]
                    for offset_idx in range(indptr[token_idx]):
                        data_idx += 1
                        indices[data_idx] = (deref(column_ptr))[offset_idx].first
                        data[data_idx] = (deref(column_ptr))[offset_idx].second
                file_dir, file_prefix = os.path.split(filename)
                tmpfd, tmpfile_fn = tempfile.mkstemp(prefix=file_prefix, dir=file_dir, suffix='-numpy.npy')
                tmpfd.close()
                self._serialize(indptr, str(field_idx) + '_indptr', zipf, tmpfile_fn)
                self._serialize(indices, str(field_idx) + '_indices', zipf, tmpfile_fn)
                self._serialize(data, str(field_idx) + '_data', zipf, tmpfile_fn)
        return


Error compiling Cython file:
------------------------------------------------------------
...
            retval = 1
        return retval


    cdef int _serialize(self,
                        np.ndarray[np.uint32_t, mode="c", ndim=1]& arr,
                                                                ^
------------------------------------------------------------

/Users/pushpendrerastogi/.ipython/cython/_cython_magic_ca0109e18d242c4d2b9c0e9c5cc431b5.pyx:43:65: Reference base type cannot be a Python object

Error compiling Cython file:
------------------------------------------------------------
...
            self.field_token_doc_count.push_back(deref(vec_ptr))
        return

    cdef int update(self, int field_idx, int token_idx, int doc_idx):
        cdef pair[int,int]* topelem_ptr
        cdef vector[pair[int,int]]* ft_ptr = &(
                                            ^
------------------------------------------------------------

/Users/pushpendrerastogi/.ipython/cython/

In [38]:
%%cython --cplus
from libcpp.vector cimport vector
from libcpp.string cimport string
from libcpp.stack cimport stack
from libcpp.pair cimport pair
from cython.operator cimport dereference as deref, postincrement


cdef FieldedHitList fhl = FieldedHitList(2, [3, 4])
print fhl.field_token_doc_count
fhl.update(0, 1, 234)
fhl.update(0, 1, 234)
fhl.update(0, 1, 235)
print fhl.field_token_doc_count


[[[], [], []], [[], [], [], []]]
[[[], [(234, 2), (235, 1)], []], [[], [], [], []]]


In [65]:
with open("/tmp/tmp.pkl") as f:
    data2 = cPickle.load(f)

#fields = ["DOCNO", "DOCHDR", "names", "category", "attributes", "SimEn", "RelEn"]
#for idx, (vocab, field) in enumerate(itertools.izip(data2, fields)):
#    data2[idx] = sorted(data2[idx].iteritems(), key=lambda x: (x[1], x[0]), reverse=True)

In [1]:
print 1

1


In [67]:
data2[5]['carmela\x00'] == 2

True